#### 폐암 사망 데이터 분석
- 사용 데이터 : lung_cancer_mortality_data_large_v2.csv
- 피처/속성 : 20개
- 타겟/라벨 : survived
- 학습-방법 : 지도학습 > 분류> 이진분류
- 학습 알고리즘 : 인공신경망(ANN) -> 심층 신경망 (MLP, DNN) : 은닉층이 많은 구성
- 프레임워크 : Pytorch

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler 
from torchmetrics.classification import BinaryF1Score, BinaryConfusionMatrix, BinaryAccuracy, BinaryRecall, BinaryPrecision
from torchinfo import summary

#### 1. 데이터 불러오기

In [ ]:
data = './lung_cancer_mortality_data_large_v2.csv'
lungDF = pd.read_csv(data)
lungDF.head()

In [3]:
lungDF = lungDF.drop(['id','country'], axis=1)

In [ ]:
lungDF.info()

#### 2. 데이터 전처리

In [5]:
lungDF['age'] = lungDF['age'].astype('int')
lungDF['beginning_of_treatment_date'] = pd.to_datetime(lungDF['beginning_of_treatment_date'])
lungDF['end_treatment_date'] = pd.to_datetime(lungDF['end_treatment_date'])
lungDF['diagnosis_date'] = pd.to_datetime(lungDF['diagnosis_date'])

In [ ]:
lungDF['cancer_stage'] = lungDF['cancer_stage'].replace({'Stage I':1,'Stage II':2, 'Stage III':3, 'Stage IV':4})
lungDF['gender'] = lungDF['gender'].replace({'Male':0, 'Female':1})
lungDF['family_history'] = lungDF['family_history'].replace({'No':0, 'Yes':1})
lungDF.info()

In [ ]:
lungDF['smoking_status'].value_counts()

In [ ]:
lungDF['treatment_type'].value_counts()

In [9]:
lungDF = pd.get_dummies(lungDF,columns=['smoking_status','treatment_type'], dtype=int)

In [ ]:
lungDF['start_days'] = (lungDF['beginning_of_treatment_date']-lungDF['diagnosis_date']).dt.days
lungDF['treatment_days'] = (lungDF['end_treatment_date'] - lungDF['beginning_of_treatment_date']).dt.days
lungDF.head()

In [ ]:
cancerDF = lungDF.drop(['diagnosis_date','beginning_of_treatment_date','end_treatment_date'], axis=1)
cancerDF.info()

3. 모델 클래스 설계 및 정의
- - -
- 클래스 목적 : lungcancer 데이터 학습 및 추론 목적
- 클래스 이름 : CancerModel
- 부모 클래스 : nn.Module
- 매 개 변 수 : 층별 입출력 개수 고정하므로 필요 없음
- 속성 / 필드 :
- 기능 / 역할 : __init__() : 모델 구조 설정, forward() : 순방향 학습 <=오버라이딩(상속관계에서만 가능)
- 클래스 구조
    * 입력층 : 입력 20개   / 출력 1000개
    * 은닉층 : 입력 1000개 / 출력 500개
    * 은닉층 : 입력 500개  / 출력 200개
    * 은닉층 : 입력 200개  / 출력 100개
    * 은닉층 : 입력 100개  / 출력 50개
    * 출력층 : 입력 50개  / 출력 1개(이진분류)
- - -
- 손실함수 / 활성화 함수
    * 클래스 형태 ==> nn.BCELoss, nn.leakyReLU ==> __init__() 메서드
    * 함수 형태 ==> torch.nn.fuctional 아래에 ==> forward() 메서드

In [12]:
class CancerModel(nn.Module):
    def __init__(self,in_out,perceptrons = []) :
        super().__init__()
        self.i_layer = nn.Linear(20,perceptrons[0] if len(perceptrons)>0 else in_out)
        
        self.h_layers = nn.ModuleList()
        for idx in range(len(perceptrons)-1) :
            self.h_layers.append(nn.Linear(perceptrons[idx], perceptrons[idx+1]))

        self.o_layer = nn.Linear(perceptrons[-1] if len(perceptrons)>0 else in_out,1)

    def forward(self, x):
        # 입력층
        y = F.leaky_relu(self.i_layer(x))

        # 은닉층
        for layer in self.h_layers:
            y = F.leaky_relu(layer(y))
        
        # 출력층
        return F.sigmoid(self.o_layer(y))


In [ ]:
model = CancerModel(5,[400,200,100,50,20,10,5])
print(model)

In [ ]:
# 모델 사용 메모리 정보 확인
summary(model, input_size=(1000000,20))

4. 데이터셋 클래스 설계 및 정의
- - -
- 데이터셋 : lung_cancer_mortality_data_large_v2.csv
- 피처 개수 : 20개
- 타겟 개수 : 1개
- 클래스 이름 : CancerDataset
- 부모 클래스 : utils.data.Dataset
- 속성 / 필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드
    * _ _init_ _(self)
    * _ _len_ _(self)
    * _ _getitem_ _(self, index) : 특정 인덱스의 피처와 타겟 반환

In [15]:
class CancerDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        return featureTS, targetTS

In [ ]:
## 데이터셋 인스턴스 생성
# 피처와 타겟 추출
featureDF = cancerDF.drop('survived',axis=1)
targetDF = cancerDF[['survived']]

# 커스텀 데이터셋 인스턴스 생성
cancerDS = CancerDataset(featureDF, targetDF)

# 데이터 로더 인스턴스 생성
cancerDL = DataLoader(cancerDS)
for feature, target in cancerDL:
    print(feature.shape, target.shape, feature, target, sep='\n')
    break

5. 학습 준비

In [17]:
### 학습 진행 관련 설정
EPOCH = 5
BATCH_SIZE = 1000
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [18]:
# 모델 인스턴스 생성
model = CancerModel(5,[32,64,100,50,20,10]).to(DEVICE)

In [ ]:
# 학습/검증/테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=17, stratify=targetDF)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=17, stratify=y_train)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')

# 학습/검증/테스트용 데이터셋 생성
trainDS = CancerDataset(X_train, y_train)
valDS = CancerDataset(X_val, y_val)
testDS = CancerDataset(X_test, y_test)

# 학습용 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE)

In [ ]:
# 최적화 인스턴스
optimizer = optim.Adam(model.parameters(), lr=LR)

# 최적화 스케줄링 인스턴스 생성
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',patience=5, verbose=True)

# 손실함수 인스턴스 : BCELoss / 예측값은 확률값으로 전달 ==> sigmoid() AF 처리 후 전달
reqLoss = nn.BCELoss()

6. 학습 진행

In [ ]:
# 학습 효과 확인
Loss_History, Score_History = [[],[]],[[],[]]
CNT = cancerDS.n_rows/BATCH_SIZE

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치크기만큼 데이터 로딩 후 학습 진행
    total_loss, total_score = 0,0

    for featureTS, targetTS in trainDL:
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = reqLoss(pre_y,targetTS)
        total_loss += loss.item()

        # 성능평가 계산
        score = BinaryF1Score()(pre_y, targetTS)
        total_score += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증
    # 검증 모드로 모델 설정
    model.eval()

    with torch.no_grad():
        # 검증용 데이터셋 생성
        val_feature_TS = torch.FloatTensor(valDS.featureDF.values)
        val_target_TS = torch.FloatTensor(valDS.targetDF.values)

        # 평가
        pre_val = model(val_feature_TS)
        # 손실 계산
        loss_val = reqLoss(pre_val, val_target_TS)
        # 성능 평가
        score_val = BinaryF1Score()(pre_val, val_target_TS)
    
    # 에포크 당 손실과 성능 평가 값 저장
    Loss_History[0].append(total_loss/CNT)
    Score_History[0].append(total_score/CNT)
    Loss_History[1].append(loss_val)
    Score_History[1].append(score_val)

    print(f'[{epoch}/{EPOCH}]\n- [TRAIN] LOSS : {Loss_History[0][-1]} SCORE : {Score_History[0][-1]}')
    print(f'- [VALID] LOSS : {Loss_History[1][-1]} SCORE : {Score_History[1][-1]}')

    # 최적화 스케줄러 인스턴스 업데이트
    scheduler.step(score_val)
    print(f'scheduler.num_bad_epochs => {scheduler.num_bad_epochs}')

    # 손실감소(또는 성능개선)가 안되는 경우 조기 종료
    if scheduler.num_bad_epochs >= scheduler.patience :
        print(f'성능 개선이 없어서 {scheduler.patience} EPOCH에 조기 종료함!')
        break